## Steps
- load datasets
- make dataset iterable
- create model class
- __instantiate model class__
- instantiate loss class
- instantiate optimizer class
- train model

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
train_dataset=dsets.MNIST(root="./data",
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

In [3]:
test_dataset=dsets.MNIST(root="./data",
                          train=False,
                          transform=transforms.ToTensor()) # important: ToTensor() not ToTensor, with the ()

In [4]:
iter=3000
batch_size=100
num_epochs=iter/(len(train_dataset)/batch_size)
num_epochs

train_gen=torch.utils.data.DataLoader(dataset=train_dataset,
                                     batch_size=batch_size,
                                     shuffle=True)

test_gen=torch.utils.data.DataLoader(dataset=test_dataset,
                                     batch_size=batch_size,
                                     shuffle=False)

In [5]:
class FeedForwardNNmodel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedForwardNNmodel, self).__init__()
        # linear function
        self.fc1=nn.Linear(input_dim, hidden_dim)
        # non-linearity
        # self.sigmoid=nn.Sigmoid()
        self.tanh=nn.Tanh()
        # Linear function
        self.fc2=nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        out=self.fc1(x)
        out=self.tanh(out)
        out=self.fc2(out)
        return out
    

## instantiate model class
- input dimension: 784
- output dimension: 10 (10 classes: from 0-9 digits)
- hidden dimension: 100
    - number of neurons
    - number of non-linear activation functions

In [6]:
input_dim=28*28
hidden_dim=100
output_dim=10

model=FeedForwardNNmodel(input_dim, hidden_dim, output_dim)

## instantiate loss class

In [7]:
criterion=nn.CrossEntropyLoss()

## instantiate optimizer class

In [8]:
optimizer=torch.optim.SGD(model.parameters(), lr=0.1) # learning rate=0.1

## train the model

In [9]:
iter=0
for epoch in range(int(num_epochs)): # epoches=5
    for images, labels in train_gen: # len(train_gen=600)
        
        # convert inputs to Variables
        images=Variable(images.view(-1, 28*28).cuda())
        labels=Variable(labels.cuda())
         
        # clear grad buffers
        optimizer.zero_grad()
        
        # get outputs with inputs
        outputs=model(images)
        
        # loss
        loss=criterion(outputs, labels)
        
        # backward gradients
        loss.backward()
        
        # updating parameters
        optimizer.step()
        
        # 1 iter: forward + backward pass
        iter+=1
        
        if iter % 500==0:
            correct=0
            total=0
            for images, labels in test_gen:
                images=Variable(images.view(-1, 28*28).cuda())
                outputs=model(images)
                # images.data.size(): (100, 784)
                # outputs.data.size(): (100, 10)
                _, predicted=torch.max(outputs.data, 1)
                
                total+=labels.size(0)
                
                correct+=(predicted.cuda()==labels.cuda()).sum()
            accuracy=correct/total*100
            print("Iteration: {}, Loss: {}, Accurracy: {}".format(iter, loss.data[0], accuracy))

RuntimeError: cuda runtime error (30) : unknown error at /pytorch/torch/lib/THC/THCGeneral.c:70